In [73]:
load("../Imports/GeneralMethods.sage", '../Imports/Utility.sage', '../Imports/Point.sage', '../Imports/Line.sage', '../Imports/TritangentPlane.sage', '../Imports/Group.sage', '../Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
xx = QQ['xx'].0
Q.<i_rad3> = NumberField(xx^2+3)

In [3]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f) * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [4]:
# Define cubic SE18, find and classify the lines and compute the 45 tritangent planes

In [5]:
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)
%time SE18 = general_cubic.subs({b:-(c*c+e*f)/c, d:(3*e*f - c^2 + c*f + c*e)/(2*c)}).subs({f:c*(3*c-e)/(c+e)}).subs({e : i_rad3*c})

CPU times: user 4.39 s, sys: 40.1 ms, total: 4.43 s
Wall time: 4.47 s


In [6]:
# Find simmetries

In [7]:
%time adm_SE18 = SE18.find_admissible_projectivities()
len(adm_SE18)

CPU times: user 1.9 s, sys: 116 ms, total: 2.02 s
Wall time: 16.2 s


648

In [8]:
%time simm_SE18 = SE18.find_simmetries(adm_SE18)
len(simm_SE18)

CPU times: user 1.59 s, sys: 161 ms, total: 1.75 s
Wall time: 2.38 s


648

In [9]:
# Study how the simmetries permute the Eckardt points

In [10]:
Eck_perms = [Permutation(SE18.apply_proj_to_eck(proj[0])).to_permutation_group_element() for proj in simm_SE18]
Eck_perms_group = PermutationGroup(Eck_perms)
Eck_perms_group.order(), Eck_perms_group.structure_description()

(648, '(((C3 x C3 x C3) : (C2 x C2)) : C3) : C2')

In [11]:
### geometric interpretation of the group of symmetries

In [12]:
%%time
## Action of the stabilizer of SE18 on the 27 lines:
Ga = perm_group_lines(SE18, simm_SE18)

## Action of the stabilizer of SE18 on the 45 tritangent planes:
Gb = perm_group_planes(SE18, simm_SE18)

CPU times: user 4min 7s, sys: 2.29 s, total: 4min 9s
Wall time: 4min 9s


In [13]:
print("Structure of the group Ga:")
print(PermutationGroup(Ga).structure_description())

print("")

print("Structure of the group Gb:")
print(PermutationGroup(Gb).structure_description())

Structure of the group Ga:
(((C3 x C3 x C3) : (C2 x C2)) : C3) : C2

Structure of the group Gb:
(((C3 x C3 x C3) : (C2 x C2)) : C3) : C2


In [14]:
## Action of the group Ga on the lines: computation of the orbits:
OGa = PermutationGroup(Ga).orbits()

## Action of the group on Gb the planes: computation of the orbits:
OGb = PermutationGroup(Gb).orbits()

In [121]:
def a(c):
    return sorted(OGb[1]).index(c)+1

In [136]:
# Define special permutations
g1 = [(a(3),a(34),a(7)),(a(15),a(32),a(19)),(a(22),a(42),a(26))]
g2 = [(a(3),a(34),a(7)),(a(14),a(33),a(20)),(a(21),a(27),a(45))]
g3 = [(a(2),a(8),a(37)),(a(15),a(32),a(19)),(a(21),a(45),a(27))]
g4 = [(a(2), a(21)), (a(3), a(42)), (a(7), a(22)), (a(8), a(45)), (a(20), a(33)), (a(26), a(34)), (a(27), a(37))]
g5 = [(a(2), a(7)), (a(3), a(37), a(34), a(8)), (a(14), a(22), a(32), a(21)), (a(15), a(45), a(33), a(42)), (a(19), a(27), a(20), a(26))]
N = PermutationGroup([g1, g2, g3])
N.structure_description()

'C3 x C3 x C3'

In [137]:
N.is_normal(Eck_perms_group)

True

In [179]:
Eck_perms_group.quotient(N).structure_description()

'S4'